<a href="https://colab.research.google.com/github/DineshThumma9/silver-doodle/blob/main/fixed_LLamaIndexRAGAdv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -Uq llama-index

In [ ]:
%pip install llama-index-llms
%pip install llama-index-embeddings-huggingface
%pip install nest_asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader

%time
docs = SimpleDirectoryReader(input_dir="./data").load_data()
len(docs)

In [ ]:
import pprint
pprint.pprint(docs)

In [ ]:
docs[0].__dict__

In [ ]:
from llama_index.core import Document
from llama_index.core.schema import MetadataMode


document = Document(
    text = "This is a super-cutomization document",
    metadata = {
        "file_name" : "super_secret_document.txt",
         "category" : "tech",
        "author" : "LlamaIndex"
    },
    # excluded_embed_metadata_keys = ["file_name"],
    excluded_embed_llm_keys = ["category"],
    metadata_template = "{key}:{value}",
    metadata_separator = "\n",
    text_template="Metadata:\n{metadata_str}\n--------\nContent:\n{content}",

)


print(

     "The embedding model sees this : \n",
     document.get_content(metadata_mode=MetadataMode.EMBED) ,

)



In [ ]:
for doc in docs:

  doc.text_template = "Metadata:\n{metadata_str}\n-----\nContent:\n{content}"

  if "page_label" not in doc.excluded_embed_metadata_keys:
    doc.excluded_embed_metadata_keys.append("page_label")

In [ ]:
print(docs[0].get_content(metadata_mode=MetadataMode.EMBED))

In [ ]:
%pip install -Uq llama-index-llms-groq

In [ ]:
from llama_index.llms.groq import Groq
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Groq API Key: ")

In [ ]:
llm = Groq(model="qwen-2.5-32b" , api_key=os.environ["GROQ_API_KEY"])

In [ ]:
from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor
)

from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(
    separator=" ",chunk_size = 1024 , chunk_overlap = 128
)


title_extractor = TitleExtractor(llm = llm ,nodes = 5)
qa_extractor = QuestionsAnsweredExtractor(llm = llm,questions=3)

from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations = [
        text_splitter,
        title_extractor,
        qa_extractor

    ]

)


nodes = pipeline.run(
    documents = docs,
    in_place = True,
    show_progress = True
)

In [ ]:
pprint.pprint(docs)

In [ ]:
pprint.pprint(docs[0].__dict__)